# Import Libraries

In [1]:
import pandas as pd

In [2]:
from multiprocessing import cpu_count

In [3]:
import string
import os

In [4]:
%%time 
import spacy
nlp = spacy.load('en')

CPU times: user 3.59 s, sys: 1.86 s, total: 5.46 s
Wall time: 3.01 s


In [5]:
base_directory = os.path.join('.','store')

In [6]:
data_directory = os.path.join('..','..','data')

Example

In [7]:
spacy.displacy.render(nlp('South African President Cyril Ramaphosa treaded a fine line between his ruling party’s rival factions when he named steady hands to key cabinet posts and purged some of his predecessor Jacob Zuma’s most ineffectual appointees.  While keeping a handful of Zuma loyalists in largely minor positions, Ramaphosa brought back as finance minister Nhlanhla Nene, who Zuma replaced in late 2015 in a decision that caused chaos in the markets. Pravin Gordhan, who won over investors during two stints in the same job before running afoul of Zuma, will oversee the six of the biggest state companies that are mostly financially strapped and mired in graft allegations.') , style='ent', minify=True, jupyter=True)

# Lemmatization Pipeline

In [8]:
data_file = os.path.join(data_directory,'sens_data.jsonlines')

In [9]:
#sens_sample = pd.read_hdf('./data/test/big_newsdata.h5', 'sens', mode='a')
sens_data = pd.read_json(data_file,convert_dates=True,lines=True,chunksize=1)

In [10]:
data_file

'../../data/sens_data.jsonlines'

In [11]:
lemma_file = os.path.join(base_directory,'json_lines_lemmas.txt')

In [139]:
def is_punct_helper(token):
    """helper function to eliminate tokens that are pure punctuation or whitespace"""
    return token.is_punct or token.is_space or token.like_num or token.is_stop or token.like_url or token.like_email


def lemmatized_sentence_corpus(generator):    
    """generator function to use spaCy to parse reviews, lemmatize the text, and yield sentences"""  
    pipe = nlp.pipe(map(lambda x: str(x['text'].values),generator),batch_size=1, n_threads=cpu_count(),disable=['ner','tag','dep'], cleanup=True)
    sent = filter(lambda x: x.sents, pipe)
    lemmas = (' '.join(map(lambda x: x.lemma_ ,filter(lambda x: not (x.is_punct or x.is_space or x.like_num or x.is_stop or x.like_url or x.like_email or x.is_bracket or x.is_quote or x.is_digit), list(next(sent))[50:-120])))+'\n')
        
    with open(lemma_file, 'w', encoding='utf_8') as f:
            f.write(lemmas)

IndentationError: unexpected indent (<ipython-input-139-250e9fc9b3f6>, line 12)

In [233]:
lemmas = map(lambda x: x.lemma_ ,filter(lambda x: not (x.is_punct or x.is_space or x.like_num or x.is_stop or x.like_url or x.like_email or x.is_bracket or x.is_quote or x.is_digit), list(next(sent))[50:-120]))

In [162]:
%%time
lemmas = (' '.join(map(lambda x: x.lemma_ ,filter(lambda x: not x.is_punct or x.is_space or x.like_num or x.is_stop or x.like_url or x.like_email or x.is_bracket or x.is_quote or x.is_digit, list(next(sent))[50:-120])))+'\n')

CPU times: user 96.7 ms, sys: 80.7 ms, total: 177 ms
Wall time: 76.3 ms


In [234]:
%%time
list(lemmas)

CPU times: user 818 µs, sys: 0 ns, total: 818 µs
Wall time: 832 µs


['samquarz',
 'this',
 'statement',
 'relate',
 'cautionary',
 'announcement',
 'issue',
 'petmin',
 'on-9',
 'january',
 '2012.-shareholder',
 'advise',
 'petmin',
 'receive',
 'notification',
 'competition',
 'commission',
 'relate',
 'sale',
 'samquarz',
 'silica',
 'in',
 'annual',
 'financial',
 'statement',
 'issue',
 'september',
 'petmin',
 'announce',
 'sell',
 'samquarz',
 'thaba',
 'chueu',
 'mining',
 'pty',
 'limited',
 'thaba’',
 'r259-million',
 'subject',
 'final',
 'approval',
 'competition',
 'commission',
 'department',
 'mineral',
 'resources',
 'on',
 'january',
 'competition',
 'commission',
 'notify',
 'petmin',
 'authorize',
 'sale',
 'the',
 'commission`s',
 'decision',
 'relate',
 'strategic',
 'importance',
 'samquarz',
 'supplier',
 'producer',
 'ferrosilicon',
 'silicon',
 'metal',
 'south',
 'africa',
 'the',
 'board',
 'management',
 'petmin',
 'thaba',
 'study',
 'commission`s',
 'decision',
 'consider',
 'option',
 'consultation',
 'advisor',
 'legal',


In [61]:
nlp.pipe()

'portfolio in the newfunds collective investment scheme in securities register as such in term of the collective investment schemes control act 45 of 2002 and manage by newfunds rf)-(proprietary limited newfunds”)’-listing of additional newfunds s&p givi resources index etf participatory interests newfunds have from commencement of business today issue an additional 70,000 s&p givi resources index etf participatory interest at a price of r42.71.-after the additional listing there will be 685,944 s&p givi resources index etf participatory interest in issue.-21 december 2017-sponsor absa bank limited act through -PRON- corporate and investment bank division)-date 21/12/2017 08:00:00 produce by the jse sens department the sens service be an information dissemination service administer by the jse limited jse’).-the jse do not whether expressly tacitly or implicitly represent warrant or in any way guarantee the truth accuracy or completeness of the information publish on sen the jse -PRON- 

In [ ]:
next(sent)[11].lemma_

In [13]:
%%time

if os.path.exists(lemma_file):
        print("A lemma file already exists. ")
else:
    lemmatized_sentence_corpus(sens_data)

A lemma file already exists. 
CPU times: user 1.17 ms, sys: 599 µs, total: 1.76 ms
Wall time: 691 µs


# Phrase Modelling
Learning conbinations of tokens that together represent meaningful multi-word concepts.  
  
$\frac{count(AB)-count_{min}}{count(a)*count(b)} * N > threashold $

Where count is is the number of times a token appears in the corpus.  
N is the total size of the corpus.  
  
 Gensim is a library for statistical analysis of sentences.  

In [14]:
%%time
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

CPU times: user 356 ms, sys: 15.4 ms, total: 371 ms
Wall time: 371 ms


## Unigrams

In [15]:
unigram_sentences = LineSentence(lemma_file)

In [16]:
import itertools
for unigram_sentence in itertools.islice(unigram_sentences, 235, 240):
    print(u' '.join(unigram_sentence))
    print(" ")

## Bigrams

In [17]:
bigram_file = os.path.join(base_directory,'bigram_file.txt')

In [18]:
%%time

if os.path.exists(bigram_file):
    print("A bigram file already exists. ")
    bigram_model = Phrases.load(bigram_file)
else:
    bigram_model = Phrases(unigram_sentences)
    bigram_model.save(bigram_file)

A bigram file already exists. 
CPU times: user 1.36 ms, sys: 0 ns, total: 1.36 ms
Wall time: 1.37 ms


In [19]:
bigram_sentences_filepath = os.path.join(base_directory,
                                         'bigram_sentences_all.txt')

In [20]:
%%time
if os.path.exists(bigram_sentences_filepath):
    print("A bigram file already exists. ")
else:
    with open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:   
        for unigram_sentence in unigram_sentences:
            bigram_sentence = u' '.join(bigram_model[unigram_sentence])
            f.write(bigram_sentence + '\n')

A bigram file already exists. 
CPU times: user 122 µs, sys: 0 ns, total: 122 µs
Wall time: 129 µs


In [21]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [22]:
for bigram_sentence in itertools.islice(bigram_sentences, 235, 240):
    print(u' '.join(bigram_sentence))
    print(u'')

## Trigrams

In [23]:
trigram_model_filepath = os.path.join(base_directory,
                                      'trigram_model_all')

In [24]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if os.path.exists(trigram_model_filepath):
    print("A trigram model already exists. ")
    # load the finished model from disk
    trigram_model = Phrases.load(trigram_model_filepath)
    
else:
    trigram_model = Phrases(bigram_sentences)

    trigram_model.save(trigram_model_filepath)

A trigram model already exists. 
CPU times: user 871 µs, sys: 0 ns, total: 871 µs
Wall time: 575 µs


In [25]:
trigram_sentences_filepath = os.path.join(base_directory,
                                          'trigram_sentences_all.txt')

In [26]:
%%time

if os.path.exists(trigram_sentences_filepath):
        print("A trigram model already exists. ")
else:
    with open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for bigram_sentence in bigram_sentences:
            trigram_sentence = u' '.join(trigram_model[bigram_sentence])
            f.write(trigram_sentence + '\n')

A trigram model already exists. 
CPU times: user 604 µs, sys: 0 ns, total: 604 µs
Wall time: 363 µs


In [27]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [28]:
for trigram_sentence in itertools.islice(trigram_sentences, 235, 240):
    print(u' '.join(trigram_sentence))
    print(u'')

## LDE

In [29]:
! pip install pyldavis

In [30]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore
from gensim.corpora import Dictionary

import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle

In [31]:
trigram_dictionary_filepath = os.path.join(base_directory,
                                           'trigram_dict_all.dict')

In [32]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to learn the dictionary yourself.
if os.path.exists(trigram_dictionary_filepath):
    print("A trigram dictionary already exists. ")
    trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)
    
else:
    # learn the dictionary by iterating over all of the reviews
    trigram_dictionary = Dictionary(trigram_sentences)
    
    # filter tokens that are very rare or too common from
    # the dictionary (filter_extremes) and reassign integer ids (compactify)
    #this was 10 and 0.4
    trigram_dictionary.filter_extremes(no_below=15, no_above=0.3)
    trigram_dictionary.compactify()

    trigram_dictionary.save(trigram_dictionary_filepath)

A trigram dictionary already exists. 
CPU times: user 746 µs, sys: 321 µs, total: 1.07 ms
Wall time: 696 µs


In [33]:
# load the finished dictionary from disk


In [34]:
trigram_bow_filepath = os.path.join(base_directory,
                                    'trigram_bow_corpus_all.mm')

In [35]:
def trigram_bow_generator(filepath):
    """generator function to read reviews from a file
    and yield a bag-of-words representation """
    
    for review in LineSentence(filepath):
        yield(trigram_dictionary.doc2bow(review))

In [36]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to build the bag-of-words corpus yourself.
if os.path.exists(trigram_bow_filepath):
        print("A trigram dictionary already exists. ")
        trigram_bow_corpus = MmCorpus(trigram_bow_filepath)
else:
    # generate bag-of-words representations for
    # all reviews and save them as a matrix
    MmCorpus.serialize(trigram_bow_filepath,
                       trigram_bow_generator(trigram_sentences_filepath))
  

A trigram dictionary already exists. 
CPU times: user 620 µs, sys: 266 µs, total: 886 µs
Wall time: 608 µs


In [37]:
  
# load the finished bag-of-words corpus from disk


In [38]:
lda_model_filepath = os.path.join(base_directory, 'lda_model_all')

In [39]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the LDA model yourself.
if os.path.exists(lda_model_filepath):
        print("A lda model already exists. ")
else:
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lda = LdaMulticore(trigram_bow_corpus,
                           num_topics=20,
                           id2word=trigram_dictionary,
                           workers=(cpu_count-1))
    
    lda.save(lda_model_filepath)

/home/marcussky/anaconda3/envs/indigo/lib/python3.6/site-packages/_pytest/fixtures.py:842: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  params = attr.ib(convert=attr.converters.optional(tuple))
/home/marcussky/anaconda3/envs/indigo/lib/python3.6/site-packages/_pytest/fixtures.py:844: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  ids = attr.ib(default=None, convert=_ensure_immutable_ids)


TypeError: unsupported operand type(s) for -: 'method' and 'int'

In [40]:
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

FileNotFoundError: [Errno 2] No such file or directory: './store/lda_model_all'

In [ ]:
def explore_topic(topic_number, topn=10):
    """accept a user-supplied topic number and
    print out a formatted list of the top terms"""
        
    print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')

    for term, frequency in lda.show_topic(topic_number, topn=25):
        print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))

In [ ]:
explore_topic(topic_number=3)

Time to name the topics

In [ ]:
topic_names_filepath = os.path.join(base_directory, 'topic_names.pkl')

In [ ]:
if os.path.exists(topic_names_filepath):
    print("A topic names pickle already exists. ")
    with open(topic_names_filepath, 'rb') as f:
        topic_names = pickle.load(f)

else:
    topic_names = {0: u'mexican',
               1: u'menu',
               2: u'thai',
               3: u'steak',
               4: u'donuts & appetizers',
               5: u'specials',
               6: u'soup',
               7: u'wings, sports bar',
               8: u'foreign language',
               9: u'las vegas',
               10: u'chicken'
               }
    with open(topic_names_filepath, 'wb') as f:
        pickle.dump(topic_names, f)

In [ ]:
LDAvis_data_filepath = os.path.join(base_directory, 'ldavis_prepared')


In [ ]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.

if os.path.exists(LDAvis_data_filepath):
    print("A LDA davis file pickle already exists. ")
    # load the pre-prepared pyLDAvis data from disk
    with open(LDAvis_data_filepath, 'rb') as f:
        LDAvis_prepared = pickle.load(f)
        
else:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus,
                                              trigram_dictionary, n_jobs=-1)

    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

Time to visualize the model

In [ ]:
pyLDAvis.enable_notebook(local=False)

In [ ]:
pyLDAvis.display(LDAvis_prepared)

In [ ]:
LDAvis_visual_output = os.path.join(base_directory, 'ldavis_visual.html')

In [ ]:
if os.path.exists(LDAvis_data_filepath):
    pass
else:
    print("A LDA visual is being written to disks. ")
    # load the pre-prepared pyLDAvis data from disk
    with open(LDAvis_visual_output, 'wb') as f:
        pyLDAvis.save_html(LDAvis_prepared, f)

In [ ]:
pyLDAvis.show(LDAvis_prepared)

## Describing LDA

# Word Vectors